###### This program create a labels based on predictions (using tflite model)

In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os, glob
from tqdm import tqdm   
import matplotlib.pyplot as plt
from termcolor import colored
from datetime import date
import random

import warnings
warnings.filterwarnings("ignore")

C:\Users\kpangalu\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#*****Load tflite model
# https://github.com/bhattbhavesh91/tflite-tutorials/blob/master/tflite-part-2.ipynb (weights of the tflite model)
# path = '/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/Normalized_NoPreprocessing/'
path = "/Users/kpangalu/Downloads/trail/"
# fname = 'CODSpeed_bullet_NoPreprocessing_NormalizedInput_13June.tflite'
fname = 'CODSpeed_bullet_combinedchkpnt_chkpnt_June25_NEW_iter_20_acc_1.0.tflite'

tfmdl_file = path + fname
print(tfmdl_file)

/Users/kpangalu/Downloads/trail/CODSpeed_bullet_combinedchkpnt_chkpnt_June25_NEW_iter_20_acc_1.0.tflite


In [3]:
#****Load tfilte model
# with open(tfmdl_file, 'rb') as fid:
#     tflite_model = fid.read()
    
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=path + fname)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#****here are the emodel details and test data should (1,14,24,1) dimensions
print(interpreter.get_input_details())
print(interpreter.get_output_details())

[{'name': 'input_input', 'index': 0, 'shape': array([ 1, 14, 24,  1]), 'shape_signature': array([-1, 14, 24,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 15, 'shape': array([  1, 302]), 'shape_signature': array([ -1, 302]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [4]:
#***Predicting the test data sets or input datasets
pubgTestroot = "/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/Detection_false/0801_false/"
# pubgTestroot = "/Users/kpangalu/Downloads/images_26July/woutlabel/"

inpdatalist= os.listdir(pubgTestroot)
print(len(inpdatalist))

77


In [5]:
#checking the image size (Combined model input : 24x14)
ii = 0
for imgindex in tqdm(range(len(inpdatalist))):
    print(imgindex,inpdatalist[imgindex])
    imgcrop  = cv2.imread(pubgTestroot+inpdatalist[imgindex],0)
    if(imgcrop.shape[1] <= 22):
        #print(imgcrop.shape[1])
        os.remove(pubgTestroot+inpdatalist[imgindex])
        ii +=1
# print(ii)
print("Number of images: {}".format(len(inpdatalist)))
print("Number of images (24x14): {}".format(len(inpdatalist)-ii))

100%|████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 1871.91it/s]

0 number_3353_62.png
1 number_3354_52.png
2 number_3355_52.png
3 number_3356_300.png
4 number_3357_59.png
5 number_3359_6.png
6 number_3363_6.png
7 number_3364_300.png
8 number_3365_300.png
9 number_3366_300.png
10 number_3368_1.png
11 number_3371_0.png
12 number_3372_300.png
13 number_3373_300.png
14 number_3374_300.png
15 number_3376_300.png
16 number_3398_52.png
17 number_3399_300.png
18 number_3414_6.png
19 number_3415_300.png
20 number_3416_300.png
21 number_3417_300.png
22 number_3431_53.png
23 number_3432_300.png
24 number_3433_52.png
25 number_3443_300.png
26 number_3444_300.png
27 number_3456_300.png
28 number_3457_300.png
29 number_3458_300.png
30 number_3459_300.png
31 number_3460_300.png
32 number_3461_300.png
33 number_3462_300.png
34 number_3463_300.png
35 number_3489_65.png
36 number_3507_7.png
37 number_3508_300.png
38 number_3509_300.png
39 number_3510_300.png
40 number_3511_300.png
41 number_3512_300.png
42 number_3513_300.png
43 number_3514_300.png
44 number_3523_7.p

In [6]:
predictions = []
test_batches = []
test_imgs= np.zeros([len(inpdatalist),14,24,1])
# test_imgs= np.zeros([len(inpdatalist),14,24])
test_lbls= np.zeros([len(inpdatalist),1])
for imgindex in tqdm(range(len(inpdatalist))):
    #print(inpdatalist[imgindex])
    imgcrop  = cv2.imread(pubgTestroot+inpdatalist[imgindex],0)
    imgcrop = np.float32(imgcrop)
    imgcrop = np.float32(imgcrop/np.max(imgcrop))
    test_imgs[imgindex,:,:,0] = (imgcrop) 
    # test_imgs[imgindex,:,:] = (imgcrop) 
    #if you don't have label just mask these lines
    #label = inpdatalist[imgindex][0:3]
    #spindex = label.find('_')
    #if spindex > 0:
    #    label = label[:spindex]
    #test_lbls[imgindex,0] = int(label)
    
    testimg = imgcrop
    #testlbl = int(label)
    
test_imgs = np.array(test_imgs, dtype=np.float32)
#test_batches.append((test_imgs, test_lbls))

100%|████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 2631.61it/s]


In [7]:
print(len(test_batches))  
# print(test_imgs.dtype)
test_imgs_numpy = np.array(test_imgs, dtype=np.float32)
print(test_imgs_numpy.dtype, test_imgs_numpy.shape)
nimgs = int(test_imgs.shape[0])
print(nimgs)

0
float32 (77, 14, 24, 1)
77


In [8]:
#****change the dimension (test data 1614,14,24,1)
interpreter.resize_tensor_input(input_details[0]['index'], (nimgs, 14, 24,1))
interpreter.resize_tensor_input(output_details[0]['index'], (nimgs, 1))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])


interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Input Shape: [77 14 24  1]
Input Type: <class 'numpy.float32'>
Output Shape: [ 77 302]
Output Type: <class 'numpy.float32'>
Prediction results shape: (77, 302)


In [9]:
#***listing actual and predictions
print("Number of images: {}".format(tflite_model_predictions.shape[0]))
# print('Number of classes: {}'.format(tflite_model_predictions.shape[1]))

k = 0
for jj in range(len(test_lbls)):
    if(test_lbls[jj] != prediction_classes[jj]):
        k += 1
        print(colored("Original bul number: {}".format(int(test_lbls[jj])),'red'), '    ',colored("Predicted bul number: {}".format(prediction_classes[jj]),'red'))
        #print("original: {}".format(int(test_lbls[jj])),'  ',"Predicted: {}".format(prediction_classes[jj]))

print(colored("Total original: {}".format(len(test_lbls)),'red'), '    ',colored("Predicted: {}".format(len(test_lbls)-k),'red'))
from sklearn.metrics import accuracy_score
acc = accuracy_score(prediction_classes, test_lbls)

# print("Accuracy: {0}%\n".format(acc*100))
# print(f"Accuracy: \033[1m{format(acc*100)}%")

Number of images: 77
Original bul number: 0      Predicted bul number: 82
Original bul number: 0      Predicted bul number: 82
Original bul number: 0      Predicted bul number: 82
Original bul number: 0      Predicted bul number: 81
Original bul number: 0      Predicted bul number: 79
Original bul number: 0      Predicted bul number: 76
Original bul number: 0      Predicted bul number: 76
Original bul number: 0      Predicted bul number: 6
Original bul number: 0      Predicted bul number: 6
Original bul number: 0      Predicted bul number: 74
Original bul number: 0      Predicted bul number: 71
Original bul number: 0      Predicted bul number: 70
Original bul number: 0      Predicted bul number: 70
Original bul number: 0      Predicted bul number: 70
Original bul number: 0      Predicted bul number: 70
Original bul number: 0      Predicted bul number: 70
Original bul number: 0      Predicted bul number: 92
Original bul number: 0      Predicted bul number: 90
Original bul number: 0     

In [10]:
print(*prediction_classes)

82 82 82 81 79 76 76 6 6 74 71 70 70 70 70 70 92 90 76 6 6 74 63 63 62 51 300 30 30 30 30 30 30 30 30 85 77 7 7 7 7 7 7 7 67 300 300 66 54 54 54 54 43 43 87 87 87 87 87 87 87 87 87 95 81 81 73 300 2 83 83 83 83 83 90 90 90


In [11]:
###Create a label for each image file
outpath ="/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/Detection_false/0801_false/"
ii = 0
for filename in os.listdir(pubgTestroot):
    src_file = pubgTestroot + filename
    splt = src_file.split('_')[-1]
    splt1 = splt.split(".")[-2]
    #print(splt1)
    #collecting predictions
    num = prediction_classes[ii]
    if(np.int(num) < 10):
        numm = '00' + str(num)
    elif(np.int(num) > 9 and np.int(num) <=99):
        numm = '0' + str(num)
    elif(np.int(num) > 99):
        numm = str(num)
    
    cur_date = date.today()
    curMonthName = cur_date.strftime("%B")
    imgdate ='_real_'+str(cur_date.day) + str(curMonthName)+'_'

    rnum = random.randint(10,994500)
    ren_file = outpath+str(numm) + imgdate + str(splt1)+'_' + str(ii)+'.png'
    os.rename(src_file, ren_file)
    
    ii += 1

In [ ]:
# #*******************display of the labelled data file
# #***Predicting the test data sets or input datasets
# pubgTestroot = "/Users/kpangalu/Downloads/test/cropped1/labelled/"
# # pubgTestroot = "/Users/kpangalu/Downloads/trail/cod-bulletdata_shivanirecently/Camera/"

# inpdatalist= os.listdir(pubgTestroot)
# print(len(inpdatalist))

# ii = 0
# for imgindex in tqdm(range(len(inpdatalist))):
#     print(inpdatalist[imgindex])
#     ii +=1